In [1]:
import numpy as np
import os
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import torch.utils.data as datatorch
import torch.nn as nn
import pandas as pd
import torch
from PIL import Image
import cv2
import torch.backends.cudnn as cudnn
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [2]:
train_triplets = np.loadtxt('train_triplets.txt', dtype= 'str')
test_triplets = np.loadtxt('test_triplets.txt', dtype= 'str')

In [3]:
print(test_triplets.shape)

(59544, 3)


In [4]:
print(train_triplets.shape)
#print()
train_triplets , val_triplets = train_test_split(train_triplets, test_size = 0.1)
print(train_triplets.shape, val_triplets.shape)
half_index = np.int64((val_triplets.shape[0]-val_triplets.shape[0]%2)/2)
print(half_index)
val_labels = np.int64(np.ones((val_triplets.shape[0],)))
print(val_labels.shape)
val_triplets[half_index:, 0], val_triplets[half_index:, 1] = val_triplets[half_index:, 1], val_triplets[half_index:, 0].copy()
val_labels[half_index:] = np.int64(0)

(59515, 3)
(53563, 3) (5952, 3)
2976
(5952,)


In [5]:
train_dir = 'food/food'
train_files = os.listdir(train_dir)
test_files = os.listdir(train_dir)


class ImageTriplesSet(Dataset):
    def __init__(self , file_array, dir, mode='train', transform = None,labels =None):
        self.triple_list = list(map(tuple, file_array))
        self.mode = mode
        self.labels = labels
        self.dir = dir
        self.transform = transform
        
    def __len__(self):
        return len(self.triple_list)
    
    def __getitem__(self,idx):
        img1 = Image.open(os.path.join(self.dir, self.triple_list[idx][0] + '.jpg'))
        img2 = Image.open(os.path.join(self.dir, self.triple_list[idx][1] + '.jpg'))
        img3 = Image.open(os.path.join(self.dir, self.triple_list[idx][2] + '.jpg'))
        
        
        if self.transform is not None:
            img1 = self.transform(img1).numpy()
            img2 = self.transform(img2).numpy()
            img3 = self.transform(img3).numpy()
        if self.labels is None:
            return img1, img2, img3
        else:
            return img1, img2, img3, self.labels[idx]
            
        #concat_img = cv2.hconcat([img1, img2, img3]).astype('float32')
        #if self.mode == 'train':
            #label = self.labels[idx]
            #return concat_img , label
            
        #else:
            #return concat_img, int(self.triple_list[idx][:-4])
        
#data_transform = transforms.Compose([
  #  transforms.Resize(350,240),
  #  transforms.CenterCrop(240),
  #  transforms.ToTensor()
#])

data_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[
                             0.229, 0.224, 0.225])
    ])

train_dataset = ImageTriplesSet(train_triplets, train_dir, transform = data_transform, labels = None)
val_dataset = ImageTriplesSet(val_triplets, train_dir, transform= data_transform, labels = None)
test_dataset = ImageTriplesSet(test_triplets, train_dir, mode="test" ,transform = data_transform,labels = None)

In [6]:
model = torch.hub.load('pytorch/vision', 'resnet18', pretrained=False)

Using cache found in C:\Users\user/.cache\torch\hub\pytorch_vision_master


In [7]:
learning_rate = 0.001
batch_size = 32
epochs = 10
logstep = int(1000 // batch_size)

train_loader = datatorch.DataLoader(dataset=train_dataset, 
                         shuffle=True, 
                         batch_size=batch_size)

#test_loader = datatorch.DataLoader(dataset=test_dataset, shuffle = False, batch_size= batch_size)



#model.fc = nn.Sequential(nn.Linear(model.fc.in_features,512),
                                  #nn.ReLU(),
                                  #nn.Dropout(),
                                  #nn.Linear(512, 2))
            
model.fc = nn.Sequential(nn.Linear(model.fc.in_features,512),
                                  #nn.ReLU(),
                                  
                                  nn.Linear(512, 4096))

#net = TripletNet(resnet101())
           
            

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cuda:0")
model =model.to(device)
#net = torch.nn.DataParallel(net).cuda()
#cudnn.benchmark = True
 #create optimizer
criterion = nn.TripletMarginLoss(margin=1.0, p=2)
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate,momentum=0.9,weight_decay=1e-5,nesterov=True)

#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='min',factor=0.1,patience=10,verbose=True)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[500,1000,1500], gamma=0.5)

training_loss_vec = []
training_accuracy_vec = []
val_f1_score = []
    
#criterion = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[500,1000,1500], gamma=0.5)

    
    
    

# loop over epochs
model.train()
for e in range(epochs):
    training_loss = 0.
    training_accuracy = 0.
    for idx, (data1, data2, data3) in enumerate(train_loader):
    #for idx, (img,label) in enumerate(train_loader):
        data1, data2, data3 = data1.cuda(), data2.cuda(), data3.cuda()
        #img, label = img.cuda(), label.cuda()
        #embedded_a, embedded_p, embedded_n = net(data1, data2, data3)
        embedded_a, embedded_p, embedded_n = model(data1), model(data2), model(data3)
        loss = criterion(embedded_a, embedded_p, embedded_n)
        # call optimizer.zero_grad()
        optimizer.zero_grad()
        # compute predictions using model
        #y_pred =  model(img)
        # compute loss
        
        #loss = criterion(y_pred,label)
        # run backward method
        loss.backward()
        # run optimizer step
        optimizer.step()
        #scheduler.step()  ######
        # logging (optional)
        
        training_loss += loss.item()
        #y_pred_idx = torch.max(y_pred.detach().cpu(),dim=1)[1]
        #training_accuracy += torch.mean((y_pred_idx == label.cpu()).float()).item()
        if (idx+1) % logstep == 0: 
            training_loss_vec.append(training_loss/logstep)
            #training_accuracy_vec.append(training_accuracy/logstep)
            print('training loss: ', training_loss/logstep)
            training_loss, training_accuracy = 0.,0.
   





training loss:  1.0439535802410496
training loss:  0.9742032385641529
training loss:  0.9669685671406407
training loss:  0.9215988305307203
training loss:  0.9342318138768596
training loss:  0.9164853634372834
training loss:  0.9021179906783565
training loss:  0.8990504087940339
training loss:  0.8985250265367569
training loss:  0.8864463663870289
training loss:  0.9228739296236346
training loss:  0.9219366042844711
training loss:  0.8444402717774914
training loss:  0.918601228344825
training loss:  0.8984927477375153
training loss:  0.8443355406484296
training loss:  0.9126397294382895
training loss:  0.930909110653785
training loss:  0.895757669402707
training loss:  0.864380951850645
training loss:  0.8549526314581594
training loss:  0.9041518934311406
training loss:  0.8641650003771628
training loss:  0.8648894909889467
training loss:  0.815254763249428
training loss:  0.8037496709054516
training loss:  0.8735001664007863
training loss:  0.8397765967153734
training loss:  0.8191075

training loss:  0.1472509955206225
training loss:  0.1472458839416504
training loss:  0.1489157844935694
training loss:  0.17464683084718644
training loss:  0.15480248870388155
training loss:  0.16833887369401992
training loss:  0.1591385708701226
training loss:  0.1752742620245103
training loss:  0.1498092908051706
training loss:  0.14790145331813442
training loss:  0.1598321146542026
training loss:  0.15597826098242112
training loss:  0.14232775040211215
training loss:  0.16181913066294887
training loss:  0.16922828170561022
training loss:  0.17537445550964725
training loss:  0.1774085255399827
training loss:  0.1668222181258663
training loss:  0.14262678305948934
training loss:  0.1587177707302955
training loss:  0.14373306353246013
training loss:  0.18217043722829512
training loss:  0.14096034486447612
training loss:  0.1579858987562118
training loss:  0.1550395426250273
training loss:  0.16542220788617287
training loss:  0.16333732874162735
training loss:  0.1778655162742061
train

training loss:  0.05348270506628098
training loss:  0.05904828108126117
training loss:  0.055440248500916264
training loss:  0.048871613317920316
training loss:  0.05051969087892963
training loss:  0.06371758205275382
training loss:  0.04159928233392777
training loss:  0.046632695101922556
training loss:  0.05884110014284811
training loss:  0.04670221670981376
training loss:  0.0678715830849063
training loss:  0.05574942356155765
training loss:  0.05188879466825916
training loss:  0.04355130849346038
training loss:  0.04859687339875006
training loss:  0.06475923715099212
training loss:  0.05184509388862118
training loss:  0.05489064512714263
training loss:  0.058021285360859286
training loss:  0.05679726552578711
training loss:  0.07479701743971917
training loss:  0.06662307247038811
training loss:  0.06471672173469298
training loss:  0.04382450734415362
training loss:  0.05842236741896598
training loss:  0.05093299861877195
training loss:  0.032458855259803035
training loss:  0.034101

In [8]:
val_loader = datatorch.DataLoader(dataset=val_dataset, shuffle = False, batch_size= 1)

val_labels_pred = []
model.eval()
for idx, (data1, data2, data3) in enumerate(val_loader):
    data1, data2, data3 = data1.cuda(), data2.cuda(), data3.cuda()
    embedded_1, embedded_2, embedded_3 = model(data1), model(data2), model(data3)
    if torch.dist(embedded_1,embedded_3,2)>=torch.dist(embedded_1,embedded_2,2):
        val_labels_pred.append(1)
    else:
        val_labels_pred.append(0)

f1 = f1_score(val_labels_pred, val_labels)
print(f1)

0.6763883825495199


In [11]:
#for i in range(0, len(val_labels_pred)): 
    #val_labels_pred[i] = int(val_labels_pred[i]) 
#f1 = f1_score(val_labels_pred, val_labels)

print(len(val_labels_pred,))

5952


In [12]:
test_loader = datatorch.DataLoader(dataset=test_dataset, shuffle = False, batch_size= 1)

In [14]:
test_triplets_pred = []
model.eval()
for idx, (data1, data2, data3) in enumerate(test_loader):
    data1, data2, data3 = data1.cuda(), data2.cuda(), data3.cuda()
    embedded_1, embedded_2, embedded_3 = model(data1), model(data2), model(data3)
    if torch.dist(embedded_1,embedded_3,2)>=torch.dist(embedded_1,embedded_2,2):
        test_triplets_pred.append(str(1))
    else:
        test_triplets_pred.append(str(0))
    

In [15]:
print(len(test_triplets_pred))
print(str(1))

59544
1


In [16]:
with open('submission2_Ketzel.txt', 'w') as f:
    for item in test_triplets_pred:
        f.write(item + '\n')